# Task1 

## Reinforcement learning can be used in the field of health care

Treatment for chronic diseases including cancer can be a long lasting process. 
Patients usually needs to have differnt kinds of treatment when they have various health conditions. 
Suppose that the treatment includes the usage of several appropriate doses of medication which can help release the pain or slow down the process of cancer.
On the other hand, improper combination or dosage of drugs may worsen the patient's health condition.
To form a Markov Decision Process. 
The state of this problem can be formulated as 

i) the concentration of tumor markers inside the blood ranging from 0 to 10 

ii) the subjective uncomfortable feeling of the patient range from 0 to 10

iii) the previous max usage of drug B

The action space is the usage of two drugs


drug A has the effect of decreasing the concentration of tumor markers while making the patinent uncomfortable, the usage of drug A starts from 0 to 9.

drug B has the effect of making the patinents less painful, but after a few times of usage. It becomes useless. The usage of drug B starts from 0 to 9 and can only be added once someone starts to use.

The assumption is that this kind of cancer will eventually lead the patient to the end of life. The final objective of the treatment is to extend the life of the patient while making the patient feel less painful. 

Given the final objective, if the patient can live one day more, there will be a constant reward of one. 

And there will also be some penalty due to the bad feeling of the patient.

To more specfic, if nothing is done, the concentration of tumor markers will increase by 10% every day. If taken one dose of drug A, the speed of the rise of the concentration of tumor markers will decrease by 1%. Howerver, the side effect is that it will make the patient feel uncomfortable by 1%. Only by increasing the usage of drug B can decrease the rise of bad feelings. And once the usage of drug B is decreased or stopped, it will cause the increase of bad feelings. 

In [9]:
import numpy as np

In [38]:
action_size_for_one = 10
actions_prob = 1/action_size_for_one/action_size_for_one
state_size_concentration = 91
state_size_feeling = 91
state_size_previous_usage_drug_B = 10

In [17]:
intial_state = [1,1,0]
action_space = []
for i in range(10):
    for j in range(10):
        action_space.append([i,j])
print(action_space)

[[0, 0], [0, 1], [0, 2], [0, 3], [0, 4], [0, 5], [0, 6], [0, 7], [0, 8], [0, 9], [1, 0], [1, 1], [1, 2], [1, 3], [1, 4], [1, 5], [1, 6], [1, 7], [1, 8], [1, 9], [2, 0], [2, 1], [2, 2], [2, 3], [2, 4], [2, 5], [2, 6], [2, 7], [2, 8], [2, 9], [3, 0], [3, 1], [3, 2], [3, 3], [3, 4], [3, 5], [3, 6], [3, 7], [3, 8], [3, 9], [4, 0], [4, 1], [4, 2], [4, 3], [4, 4], [4, 5], [4, 6], [4, 7], [4, 8], [4, 9], [5, 0], [5, 1], [5, 2], [5, 3], [5, 4], [5, 5], [5, 6], [5, 7], [5, 8], [5, 9], [6, 0], [6, 1], [6, 2], [6, 3], [6, 4], [6, 5], [6, 6], [6, 7], [6, 8], [6, 9], [7, 0], [7, 1], [7, 2], [7, 3], [7, 4], [7, 5], [7, 6], [7, 7], [7, 8], [7, 9], [8, 0], [8, 1], [8, 2], [8, 3], [8, 4], [8, 5], [8, 6], [8, 7], [8, 8], [8, 9], [9, 0], [9, 1], [9, 2], [9, 3], [9, 4], [9, 5], [9, 6], [9, 7], [9, 8], [9, 9]]


In [31]:
bad_feeling_constant = 0.2

In [53]:
def step(state, action):
    drugA_usage = action[0]
    drugB_usage = action[1]
    increase_rate_tumor = 10 - drugA_usage
    increase_rate_bad_feeling = drugA_usage + state[2] - drugB_usage
    
    new_state = [0,0,0]
    new_state[0] = np.round(state[0]*(1+increase_rate_tumor/100),1)
    new_state[1] = np.round(state[1]*(1+increase_rate_bad_feeling/100),1)
    new_state[0] = min(10, new_state[0])
    new_state[1] = min(10, new_state[1])
    new_state[0] = max(1, new_state[0])
    new_state[1] = max(1, new_state[1])
    new_state[2] = max(drugB_usage, state[2])
    
    reward = 1
    reward -= bad_feeling_constant * new_state[1]
    done = False
    if ((new_state[1] >= 10) or (new_state[0] >= 10)):
        done = True
    return new_state, reward, done

In [58]:
def get_index(state):
    index = int((state - 1)/0.1)
    if index == 91:
        print(state)
    return int((state - 1)/0.1)

In [59]:
def compute_state_value(max_iter=9, discount=1.0, policy = actions_prob*np.ones((state_size_concentration,state_size_feeling,state_size_previous_usage_drug_B,action_size_for_one*action_size_for_one))):
    new_state_values = np.zeros((state_size_concentration,state_size_feeling,state_size_previous_usage_drug_B))
    iteration = 0
    
    
    while iteration<=max_iter:
      state_values = new_state_values.copy()
      old_state_values = state_values.copy()

      for i in np.linspace(1,10,state_size_concentration):
          for j in np.linspace(1,10,state_size_feeling):
              for m in range(state_size_previous_usage_drug_B):
                  i =  np.round(i,1)
                  j =  np.round(j,1)
                  index_i = get_index(i)
                  index_j = get_index(j)
                  value = 0
                  for k,a in enumerate(actions):
                      (next_i, next_j,next_m), reward, done = step([i, j, m], a)
                      next_index_i = get_index(next_i)
                      next_index_j = get_index(next_j)
                      value += policy[index_i,index_j,m,k] * (reward + discount * state_values[next_index_i, next_index_j,next_m])
                  new_state_values[index_i, index_j, m] = value

      iteration += 1 
      
    return new_state_values, iteration

In [60]:
values, sync_iteration = compute_state_value(max_iter=3)
print(values)

[[[ 1.89518589  1.89051758  1.88414297 ...  1.82123657  1.80573438
    1.78989006]
  [ 1.78969354  1.7835197   1.77595129 ...  1.71220399  1.69641681
    1.67926137]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [-2.36466777 -2.37998596 -2.39308752 ... -2.42562194 -2.42719605
   -2.42780821]
  [-2.37908742 -2.39164364 -2.4019108  ... -2.4268671  -2.42780619
   -2.42800821]
  [-2.39077331 -2.40048975 -2.40846517 ... -2.42748137 -2.42800821
   -2.42800821]]

 [[ 1.89518589  1.89051758  1.88414297 ...  1.82123657  1.80573438
    1.78989006]
  [ 1.78969354  1.7835197   1.77595129 ...  1.71220399  1.69641681
    1.67926137]
  [ 0.          0.          0.         ...  0.          0.
    0.        ]
  ...
  [-2.36466777 -2.37998596 -2.39308752 ... -2.42562194 -2.42719605
   -2.42780821]
  [-2.37908742 -2.39164364 -2.4019108  ... -2.4268671  -2.42780619
   -2.42800821]
  [-2.39077331 -2.40048975 -2.40846517 ... -2.42748137 -2.42800821
   -2.42800821]]

 [[ 

## Reinforcement learning can be used in parking

We can teach the agent car to park itself.
The state spaces include:

i) The angle of the car

ii) The velocity of the car

iii) the position x of the car

v) the position y of the car

all the angles and positions are in global coordinate

The objective of the agent is to park the car into position x_tar and y_tar within a error of err_x and err_y

The reward function is:

After one step, there will be a constant reward of -1.

If the car crash or get into infeasible region, the reward is -100 and this episode ends.

If the car get into the right place, the reward is 1000 this episode ends

The action space:

The agent can choose to accelerate or deaccelerate with the value ranging from -1 to 1 m/s^2.

The agent can also choose to change the angle of the car with the angular aceeleration from -90 to 90 degree/s.

Every step, the position of the car is updated based on the velocity and angle. The velocity is updated based on the acceleration.